## Everything is an Object
- On *nix type operating systems, "everything is a file"; the OS exposes a set of programming calls that treat all the various parts of an operating system as though they were files. Network sockets, processes and threads, and areas of live memory are files with relative locations in a tree structure, that can be read from and written to, and have read and write permissions applied for certain users and groups
- On Windows, **everything is an Object**. This means everything in the operating system is represented in memory as an object data structure, with properties and members, and methods that collectively provide an API to that operating system component
- The _object manager_ is the component of the Windows NTOSKernel (or kernel for short) that managed these objects, their memory allocation, and lifetimes
- The kernel maintains a list of _types_ of object it supports, which can be examined with the `Get-NtType` cmdlet

In [4]:
Get-NtType | Select -First 20


Name
----
Type
Directory
SymbolicLink
Token
Session
Job
Process
Thread
Partition
UserApcReserve
IoCompletionReserve
ActivityReference
ProcessStateChange
ThreadStateChange
CpuPartition
SchedulerSharedData
PsSiloContextPaged
PsSiloContextNonPaged
DebugObject
Event



## Object Manager Namespace
- The Object Manager Namespace or OMNS for short, is a filesystem-like structure hidden in the background of the operating system
- The OMNS is comprised of objects called _directories_, which act as containers for the active _instances_ of kernel objects as they exist in memory, and which can be interacted with like files in the directory 
- Its important to note that while the directory object in the OMNS _behaves_ like a file directory, it is not representative of an actual file directory
- The [Windows driver docs state](https://learn.microsoft.com/en-us/windows-hardware/drivers/kernel/object-directories):
 
    >Do not confuse object directories with file system directories. Object directories exist only within the object manager, and do not correspond to any directory on disk. File system directories are, in fact, represented as file objects
- However, the `NtObjectManager` module contains a PSdrive provider, which abstracts the OMNS in such a way that it can be traversed as if it were a file system

In [5]:
Get-ChildItem NtObject:\* | Select -Property Name, TypeName -First 9 | Format-List


Name     : UnionfsPort
TypeName : FilterConnectionPort

Name     : PendingRenameMutex
TypeName : Mutant

Name     : ObjectTypes
TypeName : Directory

Name     : storqosfltport
TypeName : FilterConnectionPort

Name     : MicrosoftMalwareProtectionRemoteIoPortWD
TypeName : FilterConnectionPort

Name     : MicrosoftDataLossPreventionPort
TypeName : FilterConnectionPort

Name     : SystemRoot
TypeName : SymbolicLink

Name     : Win32kSiloSessionGlobals
TypeName : Section

Name     : Sessions
TypeName : Directory



- Example of how the Symbolic Link type works (it contains a pointer to another object)

In [6]:
Get-ChildItem NtObject:\Dfs | Select -Property Name, TypeName, SymbolicLinkTarget


Name TypeName     SymbolicLinkTarget
---- --------     ------------------
Dfs  SymbolicLink \Device\DfsClient



- The users and groups that have access to an object is defined in the objects _security descriptor_

In [7]:
Get-Item NtObject:\Device\DfsClient | Select -Property Name, TypeName, SecurityDescriptor


Name      TypeName SecurityDescriptor
----      -------- ------------------
DfsClient Device   O:BAG:SYD:(A;;FA;;;S-1-5-80-719998295-2833700043-1566817583-4093942769-14140263…



## System Calls
- The _object manager_ is the kernel-level component that handles objects and mediates interaction with those objects
- For a user space application to interact with a kernel object, the application needs to use the _system call interface_ 
- A system call is a function provided by one of the many DLLs that form the Windows subsystem; calling this function will call its kernel-mode equivilent and make a request to the object manager to access a specific kernel object
- For example, certain PowerShell cmdlets use _native_ code i.e rely on the functionality exposed by the win32 subsystem DLLs, such as `Get-ChildItem`

In [8]:
Get-ChildItem C:\


    Directory: C:\

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----            4/9/2025  5:00 PM                inetpub
d----           4/28/2024  1:43 PM                Intel
d----            4/1/2024  8:26 AM                PerfLogs
dar--            4/8/2025 11:06 PM                Program Files
d-r--            4/9/2025  5:00 PM                Program Files (x86)
d-r--           3/19/2025  4:27 PM                Users
d----            4/9/2025  5:00 PM                Windows
-a---           2/22/2024 12:33 AM         112136 appverifUI.dll
-a---           2/22/2024 12:34 AM          66328 vfcompat.dll



- Running this cmdlet will produce the following system calls (also just called _syscalls_). Below output was taken from **ProcMon**. 
- The process is showing as **dotnet.exe** because this notebook is being ran via the magic of the .NET interactive kernel:  

![](./Images/C2_Syscalls.png)

### What is the difference between user-space and kernel-space
- The [MS driver docs](https://learn.microsoft.com/en-us/windows-hardware/drivers/gettingstarted/user-mode-and-kernel-mode) do a good job of explaining this:

    >_"A processor in a computer running Windows operates in two different modes: **user mode** and **kernel mode**. The processor switches between these modes depending on the type of code it's executing. Applications operate in **user mode**, while core operating system components function in **kernel mode**."_ 

- When a user-mode application wants to interact with the operating system, it does so via interacting with objects held in kernel-mode memory, which are managed by the system object manager.
- Furthermore, an important thing to know is there is a distinct separation in how the code is mapped to the operating systems memory:  

    >_"When you launch an application in **user mode**, Windows creates a process for it. This process provides the application with a private virtual address space and a private handle table. Since each application's virtual address space is private, one application can't modify another application's data."_

    >_"All code running in **kernel mode** shares a single virtual address space. As a result, a kernel-mode driver isn't isolated from other drivers or the operating system. If a kernel-mode driver mistakenly writes to the wrong virtual address, it could compromise data belonging to the operating system or another driver. If a kernel-mode driver crashes, it causes the entire operating system to crash."_

## Syscall Internals
- One of the system calls used when `Get-ChildItem` executes is  **NtCreateFile**
- Functions prefixed with **Nt** involve calls from userland applications into the kernel
- The [documentation for this function](https://learn.microsoft.com/en-us/windows/win32/api/winternl/nf-winternl-ntcreatefile) describes it as doing the following:
  
  >Creates a new file or directory, or opens an existing file, device, directory, or volume.

- The definition of **NtCreateFile** is as follows:
```
__kernel_entry NTSTATUS NtCreateFile(
  [out]          PHANDLE            FileHandle,
  [in]           ACCESS_MASK        DesiredAccess,
  [in]           POBJECT_ATTRIBUTES ObjectAttributes,
  [out]          PIO_STATUS_BLOCK   IoStatusBlock,
  [in, optional] PLARGE_INTEGER     AllocationSize,
  [in]           ULONG              FileAttributes,
  [in]           ULONG              ShareAccess,
  [in]           ULONG              CreateDisposition,
  [in]           ULONG              CreateOptions,
  [in]           PVOID              EaBuffer,
  [in]           ULONG              EaLength
);
```

- The 3rd parameter of the above signature is _ObjectAttributes_, which is a pointer to the following structure:
```
typedef struct _OBJECT_ATTRIBUTES {
  ULONG           Length;
  HANDLE          RootDirectory;
  PUNICODE_STRING ObjectName;
  ULONG           Attributes;
  PVOID           SecurityDescriptor;
  PVOID           SecurityQualityOfService;
} OBJECT_ATTRIBUTES;
```

- The `RootDirectory` or `ObjectName` parameters need to be provided with the path to the file object to either create or open that object
- `RootDirectory` is meant to contain the base container object that the intended file object resides in, while `ObjectName` it the name of the object itself, alternatively the full path to the object as it exists in the OMNS can be provided in the `ObjectName` and the call will still reach the correct object

## NT Status Codes
- System calls return a 32-bit status code referred to as the _NT Status Code_ 
- The most important bits in the NT Status code are the _severity_, the _facility_, and the _status code_ itself:
    - The **status severity** tells us how significant the status is: indicating a success, being informational, a warning or an error
    - The **facility** is the component or subsystem associated with the status code. For example, _default_ is a catch all facility value, while _NtWin32_ is the facility for status messages that originate specifically within the Win32 susbsystem DLLs
    - The **status code** itself is a unique identifier within each facility that indicates the meaning of the status message
- The cmdlets in the `NtObjectManager` module make syscalls directly, and the status code returned by the syscall is exposed as a .NET exception  

In [9]:
Get-NtDirectory \NONEXISTENT

Get-NtDirectory: (0xC0000034) - Object Name not found.


Error: (0xC0000034) - Object Name not found.

- The `Get-NtStatus` command can then be used to translate the error code returned by the failed `Get-NtDirectory` invocation, and provide human readable version of the status, including the severity, facility, and decimal status code 
- Other interesting fields are the customer code; if this status was created as part of a 3rd party application and not a Windows component, this would be set to **true**

In [10]:
Get-NtStatus 0xC0000034 | Select -Property StatusSigned, StatusName, Message, Code, CustomerCode, Facility, Severity | Format-List


StatusSigned : -1073741772
StatusName   : STATUS_OBJECT_NAME_NOT_FOUND
Message      : Object Name not found.
Code         : 52
CustomerCode : False
Facility     : FACILITY_DEFAULT
Severity     : STATUS_SEVERITY_ERROR



## Handles
- The object manager namespace is a kernel component, and therefore objects it manages are stored in kernel memory, and user space programs cannot access kernel memory directly
- When user-space applications want to interact with an object, they must do so indirectly via a mechanism known as a _handle_
- Each user-space process has a corresponding kernel-space _process object_, and contained in this process object is the processes _handle table_ 
- For each resource the process interacts with, the process handle table will contain an entry which includes a handle identifier, the type of object the handle points to, what rights the handle grants, and a pointer to the resource in kernel memory
- As can be seen below, the handle table for each process can grow fairly large:

In [11]:
Get-Process dotnet | Select -Property Name, Id, Handles


Name     Id Handles
----     -- -------
dotnet 2208     282
dotnet 4300     886
dotnet 4448     574
dotnet 6700     573



- When a user-space process requests, and is granted, access to a resource, the kernel will add an entry for that resource to the requesting processes handle table, and this entry can then be reused
  - The process is said to then 'have a handle to that object'
  - Child processes of this process may then inherit the handle table
- Whenever the process makes performs a syscall that requires access to some resource, it will supply the handle identifier for the resource in question from its handle table, as a parameter to the syscall
```
__kernel_entry NTSTATUS NtCreateFile(
  [out]          PHANDLE            FileHandle,
  [in]           ACCESS_MASK        DesiredAccess,
```
- The first parameter called `FileHandle` with type `PHANDLE` (`PHANDLE` is just a typedef to `VOID`) is where the process passes the handle identifier 
- The privileges the process is requesting on the object is provided via the `DesiredAccess` parameter
- The type of the `DesiredAccess` parameter is `ACCESS_MASK`, which is the same type that gets stored in the process handle table
  - The _access mask_ is how the process defines the level of access it requires on the object, and also how the object defines the level of access a process should be allowed
- The below output of `Get-NtHandle` shows the access granted via the handle and the handle type

In [17]:
Get-NtHandle -ProcessId $(Get-Process dotnet | Select -First 1).ID | Select -First 10 


ProcessId Handle ObjectType           Object           GrantedAccess
--------- ------ ----------           ------           -------------
2208      4      File                 0000000000000000 0012019F
2208      8      Event                0000000000000000 001F0003
2208      12     Event                0000000000000000 001F0003
2208      16     IRTimer              0000000000000000 00100002
2208      20     TpWorkerFactory      0000000000000000 000F00FF
2208      24     IoCompletion         0000000000000000 001F0003
2208      28     WaitCompletionPacket 0000000000000000 00000001
2208      32     IRTimer              0000000000000000 00100002
2208      36     WaitCompletionPacket 0000000000000000 00000001
2208      40     Event                0000000000000000 001F0003



- As mentioned baove when a syscall is called, the syscall implementation takes the handle identifier passed to it, and convert this identifier to a kernel pointer
- As part of this converion the syscall will also check that the access the process has requested for this operation matches the granted access as it is recorded in the handle table (if this check fails the error returned if STATUS_ACCESS_DENIED)
- The syscall implementation also checks the handle type matches the resource type (if this fails the error returned is STATUS_OBJECT_TYPE_MISMATCH)

In [ ]:
Get-NtAccessMask -FileAccess ReadData, WriteData, AppendData, ReadEa, WriteEa, ReadAttributes, WriteAttributes, ReadControl, Synchronize